In [9]:
import pandas as pd

In [10]:
df=pd.read_csv("features_3_sec.csv")
df=df.iloc[:,1:]
df.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["encoded_label"]=le.fit_transform(df["label"])

In [12]:
import json

# Convert np.int32 → int
classes = {str(k): int(v) for k, v in zip(le.classes_, le.transform(le.classes_))}

with open("label_classes2.json", "w") as file:
    json.dump(classes, file)


In [13]:
X=df.drop(columns=["label","encoded_label"])
Y=df["encoded_label"]
X.head()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,3714.560359,...,-2.853603,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767
1,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,3869.682242,...,4.074709,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081
2,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,3997.639160,...,4.806280,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122
3,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,3568.300218,...,-1.359111,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678
4,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,3469.992864,...,2.092937,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694


In [14]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(X,Y,train_size=0.80,random_state=13)

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled=scaler.transform(x_test)

In [16]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

# Define model parameter grids
param_grid = {
    "Random Forest": {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None]
    },
    "SVM": {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale', 'auto']
    },
    "KNN": {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance']
    },
    "MLP": {
        'hidden_layer_sizes': [(128, 64), (128, 64, 32)],
        'alpha': [0.0001, 0.001],
        'activation': ['relu', 'tanh']
    }
}

# Map model names to initializers
model_initializers = {
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True),  # soft voting needs predict_proba
    "KNN": KNeighborsClassifier(),
    "MLP": MLPClassifier(max_iter=300)
}

best_models = {}

# Tune each model with GridSearch
for name, params in param_grid.items():
    print(f"\n🔍 Tuning {name}...")

    model = model_initializers[name]

    grid = GridSearchCV(model, params, cv=5, n_jobs=-1, scoring='accuracy')
    grid.fit(X_train_scaled, y_train)

    print(f"✅ Best Params: {grid.best_params_}")
    print(f"📈 Best CV Score: {grid.best_score_:.3f}")

    best_models[name] = grid.best_estimator_

# 🧠 Use top 3 models in VotingClassifier
voting_clf = VotingClassifier(
    estimators=[
        ('rf', best_models["Random Forest"]),
        ('svm', best_models["SVM"]),
        ('mlp', best_models["MLP"])
    ],
    voting='soft'  # uses probabilities
)

# Train and evaluate VotingClassifier
voting_clf.fit(X_train_scaled, y_train)
y_pred = voting_clf.predict(X_test_scaled)

print("\n🧠 Voting Classifier (Soft Voting)")
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("📊 Classification Report:\n", classification_report(y_test, y_pred))



🔍 Tuning Random Forest...
✅ Best Params: {'max_depth': None, 'n_estimators': 200}
📈 Best CV Score: 0.860

🔍 Tuning SVM...
✅ Best Params: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
📈 Best CV Score: 0.910

🔍 Tuning KNN...
✅ Best Params: {'n_neighbors': 3, 'weights': 'distance'}
📈 Best CV Score: 0.888

🔍 Tuning MLP...
✅ Best Params: {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (128, 64, 32)}
📈 Best CV Score: 0.877

🧠 Voting Classifier (Soft Voting)
✅ Accuracy: 0.9164164164164165
📊 Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.97      0.94       207
           1       0.96      0.94      0.95       203
           2       0.90      0.87      0.89       221
           3       0.90      0.90      0.90       193
           4       0.97      0.89      0.93       190
           5       0.92      0.96      0.94       217
           6       0.95      0.93      0.94       216
           7       0.92      0.92      0.